In [1]:
# !pip install bayesian-optimization
!pip install sklearn-genetic
# !pip install sklearn-deap
# !pip install shap
# !pip install git+https://github.com/jundongl/scikit-feature
# !pip install yellowbrick
#!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o
!pip install pycaret-nightly



     |████████████████████████████████| 163kB 5.1MB/s 
     |████████████████████████████████| 266kB 6.7MB/s 
     |████████████████████████████████| 1.7MB 25.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 276kB 48.2MB/s 
     |████████████████████████████████| 2.0MB 20.3MB/s 
     |████████████████████████████████| 174kB 45.2MB/s 
     |████████████████████████████████| 6.8MB 29.8MB/s 
     |████████████████████████████████| 256kB 42.9MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 1.4MB 29.7MB/s 
     |████████████████████████████████| 14.2MB 286kB/s 
     |████████████████████████████████| 112kB 54.0MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 3.1MB 15.0MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 

In [2]:
import pandas as pd 
import numpy as np 
import sklearn
from google.colab import files, drive, data_table
from google.colab.data_table import *
from lightgbm import *
from sklearn.ensemble import *
from sklearn.ensemble import *
from sklearn.neural_network import *
from sklearn.model_selection import *
from sklearn.naive_bayes import *
from xgboost import *
from sklearn.svm import *
from sklearn.linear_model import *
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import *

from sklearn.tree import *
import warnings
from sklearn.utils import *
from yellowbrick.model_selection import * 
from yellowbrick.classifier import * 
import matplotlib.pyplot as plt
#from bayes_opt import *
#from mlxtend.feature_selection import *
import warnings
warnings.filterwarnings("ignore")
#import h2o
##from h2o.automl import H2OAutoML
##h2o.init()

mcc = make_scorer(matthews_corrcoef)

# Data Preprocessing

In [3]:
files.upload()

Saving heart_failure_clinical_records_dataset.csv to heart_failure_clinical_records_dataset.csv


{'heart_failure_clinical_records_dataset.csv': b'age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT\n75,0,582,0,20,1,265000,1.9,130,1,0,4,1\n55,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1\n65,0,146,0,20,0,162000,1.3,129,1,1,7,1\n50,1,111,0,20,0,210000,1.9,137,1,0,7,1\n65,1,160,1,20,0,327000,2.7,116,0,0,8,1\n90,1,47,0,40,1,204000,2.1,132,1,1,8,1\n75,1,246,0,15,0,127000,1.2,137,1,0,10,1\n60,1,315,1,60,0,454000,1.1,131,1,1,10,1\n65,0,157,0,65,0,263358.03,1.5,138,0,0,10,1\n80,1,123,0,35,1,388000,9.4,133,1,1,10,1\n75,1,81,0,38,1,368000,4,131,1,1,10,1\n62,0,231,0,25,1,253000,0.9,140,1,1,10,1\n45,1,981,0,30,0,136000,1.1,137,1,0,11,1\n50,1,168,0,38,1,276000,1.1,137,1,0,11,1\n49,1,80,0,30,1,427000,1,138,0,0,12,0\n82,1,379,0,50,0,47000,1.3,136,1,0,13,1\n87,1,149,0,38,0,262000,0.9,140,1,0,14,1\n45,0,582,0,14,0,166000,0.8,127,1,0,14,1\n70,1,125,0,25,1,237000,1,140,0,0,15,1\n48,1,582,1,55,0,87000,1.9,1

In [4]:
D = pd.read_csv("heart_failure_clinical_records_dataset.csv")
allfeats = D.columns
allfeats = list(allfeats)
#allfeats.remove("time")
allfeats.remove("DEATH_EVENT")
allfeats.remove("time")
catcols = ["anaemia", "diabetes", "high_blood_pressure", "sex", "smoking"]
# for c in catcols:
#   D[c] = D[c].astype("category")
numcols = set(allfeats) - set(catcols)
numcols = list(numcols)
#numcols.remove("DEATH_EVENT")
print(numcols, catcols)

D["time"].describe()

['platelets', 'age', 'serum_sodium', 'ejection_fraction', 'creatinine_phosphokinase', 'serum_creatinine'] ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']


count    299.000000
mean     130.260870
std       77.614208
min        4.000000
25%       73.000000
50%      115.000000
75%      203.000000
max      285.000000
Name: time, dtype: float64

In [5]:
D[allfeats].dtypes

age                         float64
anaemia                       int64
creatinine_phosphokinase      int64
diabetes                      int64
ejection_fraction             int64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                  int64
sex                           int64
smoking                       int64
dtype: object

In [6]:
from sklearn.preprocessing import * 

for col in numcols: 
  scaler = MinMaxScaler()
  D[col] = scaler.fit_transform(D[col].values.reshape(-1,1))

In [7]:
D.iloc[:,-1] = D.iloc[:,-1].astype(np.int64)
D[allfeats]

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking
0,0.636364,0,0.071319,0,0.090909,1,0.290823,0.157303,0.485714,1,0
1,0.272727,0,1.000000,0,0.363636,0,0.288833,0.067416,0.657143,1,0
2,0.454545,0,0.015693,0,0.090909,0,0.165960,0.089888,0.457143,1,1
3,0.181818,1,0.011227,0,0.090909,0,0.224148,0.157303,0.685714,1,0
4,0.454545,1,0.017479,1,0.090909,0,0.365984,0.247191,0.085714,0,0
...,...,...,...,...,...,...,...,...,...,...,...
294,0.400000,0,0.004848,1,0.363636,1,0.157474,0.067416,0.857143,1,1
295,0.272727,0,0.229268,0,0.363636,0,0.296884,0.078652,0.742857,0,0
296,0.090909,0,0.259888,1,0.696970,0,0.869075,0.033708,0.714286,0,0
297,0.090909,0,0.304925,0,0.363636,0,0.139290,0.101124,0.771429,1,1


In [8]:
X = D[allfeats] ; y = D["DEATH_EVENT"].values
X

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking
0,0.636364,0,0.071319,0,0.090909,1,0.290823,0.157303,0.485714,1,0
1,0.272727,0,1.000000,0,0.363636,0,0.288833,0.067416,0.657143,1,0
2,0.454545,0,0.015693,0,0.090909,0,0.165960,0.089888,0.457143,1,1
3,0.181818,1,0.011227,0,0.090909,0,0.224148,0.157303,0.685714,1,0
4,0.454545,1,0.017479,1,0.090909,0,0.365984,0.247191,0.085714,0,0
...,...,...,...,...,...,...,...,...,...,...,...
294,0.400000,0,0.004848,1,0.363636,1,0.157474,0.067416,0.857143,1,1
295,0.272727,0,0.229268,0,0.363636,0,0.296884,0.078652,0.742857,0,0
296,0.090909,0,0.259888,1,0.696970,0,0.869075,0.033708,0.714286,0,0
297,0.090909,0,0.304925,0,0.363636,0,0.139290,0.101124,0.771429,1,1


# Logistic Regression Baseline

In [10]:
rkf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 2)
clf = LogisticRegression(solver = "liblinear")
X = D[allfeats] ; y = D.iloc[:,-1]
cv_score = np.mean(cross_val_score(clf, X, y, cv = rkf, scoring = mcc))
print(cv_score)

0.261474686743305


# Logistic Regression + GA

In [11]:
estimator = LogisticRegression(solver = "liblinear", C = 6, tol = 1, fit_intercept = True)

from sklearn.model_selection import *
report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 
rkf = RepeatedStratifiedKFold(n_repeats = 20, n_splits = 5)
for i in range(2,11):
  
  selector = GeneticSelectionCV(estimator,
                                cv = rkf,
                                verbose = 0,
                                scoring = mcc,
                                max_features = i,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.1,
                                #tournament_size = 3,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  selector = selector.fit(D[allfeats], y)
  genfeats = D[allfeats].columns[selector.support_]
  genfeats = list(genfeats)
  print("Chosen Feats:  ", genfeats)

  cv_score = selector.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore



Chosen Feats:   ['age', 'ejection_fraction']
Chosen Feats:   ['age', 'ejection_fraction', 'serum_creatinine']
Chosen Feats:   ['age', 'diabetes', 'ejection_fraction', 'serum_creatinine']
Chosen Feats:   ['age', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'serum_creatinine']
Chosen Feats:   ['age', 'diabetes', 'ejection_fraction', 'serum_creatinine']
Chosen Feats:   ['age', 'diabetes', 'ejection_fraction', 'serum_creatinine']
Chosen Feats:   ['age', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'serum_creatinine']
Chosen Feats:   ['age', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'serum_creatinine']
Chosen Feats:   ['age', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'serum_creatinine']


In [18]:
report["Scores"] = np.round(report["Scores"], 3)
report.sort_values(by = "Scores", ascending = False, inplace = True)
#report.index
print(report)
ga_feats = report.iloc[0]["Chosen Feats"]
DataTable(report)
print(ga_feats)

   No of Feats                                       Chosen Feats  Scores
7            3         [age, ejection_fraction, serum_creatinine]   0.828
2            4  [age, anaemia, ejection_fraction, serum_creati...   0.823
5            6  [age, creatinine_phosphokinase, ejection_fract...   0.815
8            7  [age, creatinine_phosphokinase, ejection_fract...   0.815
1            3         [age, ejection_fraction, serum_creatinine]   0.813
3            5  [age, diabetes, ejection_fraction, serum_creat...   0.813
6            8  [age, creatinine_phosphokinase, diabetes, ejec...   0.810
4            4  [age, ejection_fraction, serum_creatinine, ser...   0.804
0            2                            [age, serum_creatinine]   0.745
['age', 'ejection_fraction', 'serum_creatinine']


In [13]:
report

,No of Feats,Chosen Feats,Scores
4,4,"[age, diabetes, ejection_fraction, serum_creat...",0.409
5,4,"[age, diabetes, ejection_fraction, serum_creat...",0.409
7,5,"[age, creatinine_phosphokinase, diabetes, ejec...",0.406
6,5,"[age, creatinine_phosphokinase, diabetes, ejec...",0.405
8,5,"[age, creatinine_phosphokinase, diabetes, ejec...",0.405
3,5,"[age, creatinine_phosphokinase, diabetes, ejec...",0.403
2,4,"[age, diabetes, ejection_fraction, serum_creat...",0.401
1,3,"[age, ejection_fraction, serum_creatinine]",0.395
0,2,"[age, ejection_fraction]",0.370


# Random Forest Baseline

In [14]:
clf = RandomForestClassifier(max_depth = 3)
rskf = RepeatedStratifiedKFold(n_repeats = 10, n_splits = 10)

#clf = LGBMClassifier(n_estimators = 41, max_depth=7)

X = D[allfeats]; y = D.iloc[:,-1]
clf.fit(X,y)
print("roc auc on entire data set", roc_auc_score(y, clf.predict(X)))
print(np.mean(cross_val_score(clf, X, y, cv = rskf, scoring = "roc_auc")))
# r = LGBMClassifier() 
# r.fit(X,y)
# print(roc_auc_score(y, r.predict(X)))

roc auc on entire data set 0.7064347290640395
0.7969412698412697


# Random Forest//Genetic Algo for feature selection

In [15]:
estimator = RandomForestClassifier(max_depth = 3)
from sklearn.model_selection import *
report = pd.DataFrame()
nofeats = [] 
chosen_feats = [] 
cvscore = [] 
rkf = RepeatedStratifiedKFold(n_repeats = 1, n_splits = 10)
for i in range(2,11):
  
  selector = GeneticSelectionCV(estimator,
                                cv = rkf,
                                verbose = 0,
                                scoring = "roc_auc",
                                max_features = i,
                                n_population = 30,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                tournament_size = 1,
                                n_gen_no_change = 5,
                                caching = True,
                                n_jobs=-1)
  selector = selector.fit(D[allfeats], y)

  print(selector.support_)

  genfeats = D[allfeats].columns[selector.support_]
  genfeats = list(genfeats)
  print(genfeats)

  X = D[genfeats] ; y = D.iloc[:,-1]
  clf = selector.estimator
  
  cv_score = selector.generation_scores_[-1]
  nofeats.append(len(genfeats)) 
  chosen_feats.append(genfeats) 
  cvscore.append(cv_score)

report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = np.round(cvscore, 3)

#This took 568 seconds or 595 seconds  
#[False  True False  True  True False False  True  True False False  True]
#['anaemia', 'diabetes', 'ejection_fraction', 'serum_creatinine', 'serum_sodium', 'time']
#Average CV score:  0.8202856225930681

[ True False False False False False False  True False False False]
['age', 'serum_creatinine']
[ True False False False  True False False  True False False False]
['age', 'ejection_fraction', 'serum_creatinine']
[ True  True False False  True False False  True False False False]
['age', 'anaemia', 'ejection_fraction', 'serum_creatinine']
[ True False False  True  True False False  True False False  True]
['age', 'diabetes', 'ejection_fraction', 'serum_creatinine', 'smoking']
[ True False False False  True False False  True  True False False]
['age', 'ejection_fraction', 'serum_creatinine', 'serum_sodium']
[ True False  True False  True False  True  True  True False False]
['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']
[ True False  True  True  True False  True  True False  True  True]
['age', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'platelets', 'serum_creatinine', 'sex', 'smoking']
[ True False False False

In [19]:
print(report)

   No of Feats                                       Chosen Feats  Scores
7            3         [age, ejection_fraction, serum_creatinine]   0.828
2            4  [age, anaemia, ejection_fraction, serum_creati...   0.823
5            6  [age, creatinine_phosphokinase, ejection_fract...   0.815
8            7  [age, creatinine_phosphokinase, ejection_fract...   0.815
1            3         [age, ejection_fraction, serum_creatinine]   0.813
3            5  [age, diabetes, ejection_fraction, serum_creat...   0.813
6            8  [age, creatinine_phosphokinase, diabetes, ejec...   0.810
4            4  [age, ejection_fraction, serum_creatinine, ser...   0.804
0            2                            [age, serum_creatinine]   0.745


In [16]:
report.sort_values(by = "Scores", ascending = False, inplace = True)
report

,No of Feats,Chosen Feats,Scores
7,3,"[age, ejection_fraction, serum_creatinine]",0.828
2,4,"[age, anaemia, ejection_fraction, serum_creati...",0.823
5,6,"[age, creatinine_phosphokinase, ejection_fract...",0.815
8,7,"[age, creatinine_phosphokinase, ejection_fract...",0.815
1,3,"[age, ejection_fraction, serum_creatinine]",0.813
3,5,"[age, diabetes, ejection_fraction, serum_creat...",0.813
6,8,"[age, creatinine_phosphokinase, diabetes, ejec...",0.810
4,4,"[age, ejection_fraction, serum_creatinine, ser...",0.804
0,2,"[age, serum_creatinine]",0.745


In [17]:
for i in range(len(report)):
  featset = report.iloc[i,1] ; print(featset)
  clf = RandomForestClassifier(max_depth = 3)
  rskf = RepeatedStratifiedKFold(n_repeats = 10, n_splits = 10)
  X = D[featset]; y = D.iloc[:,-1]
  print(np.mean(cross_val_score(clf, X, y, cv = rskf, scoring = "roc_auc")))
  #.817

['age', 'ejection_fraction', 'serum_creatinine']
0.8097667989417989
['age', 'anaemia', 'ejection_fraction', 'serum_creatinine']
0.8081757936507937
['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']
0.7992923280423281
['age', 'creatinine_phosphokinase', 'ejection_fraction', 'high_blood_pressure', 'serum_creatinine', 'serum_sodium', 'sex']
0.7996661375661374
['age', 'ejection_fraction', 'serum_creatinine']
0.8105705026455027
['age', 'diabetes', 'ejection_fraction', 'serum_creatinine', 'smoking']
0.7995469576719576
['age', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'platelets', 'serum_creatinine', 'sex', 'smoking']
0.7998859788359789
['age', 'ejection_fraction', 'serum_creatinine', 'serum_sodium']
0.8023404761904761
['age', 'serum_creatinine']
0.7374715608465608
